# 1. Outline

**Bayesian Models**

- Why Bayesian？


- MLE、MAP、Bayesian Estimation  


- Bayesian Inference

    - Approximate Inference
        - MCMC
        - Variational Inference
        

- LDA Model


- Inference of LDA
    - Gibbs Sampling
    - Collapsed Gibbs Sampling
    - Variational Inference
    - Stochastic Gradient MCMC
    - Stochastic Variational Inference
    - Distributed MCMC
    
    
- Other Bayesian Models


# 2. Why Bayesian?

#### 1. LDA（Latent Dirichlet Allocation）is popular

LDA vs. k-means:  
> LDA给出是属于各个主体的概率分布；（软分类）  
k-means是硬分类。

#### 2. <font color=red>Small Data Problem</font>  

当数据量比较小时，模型容易Overfitting。集成模型可以解决Overfitting，而Bayesian Model可以看做是集成模型。

#### 3. Incorporate Uncertainty

#### 4. 先验（Incorporate Prior）

#### 5. Compression

# 3. MLE、MAP、Bayesian Estimation

<img src='attachment/69. MLE_MAP_Bayesian.png' style='zoom:50%'/>

# 4. Bayesian Inference

$$
\hat{y} = \int_{\theta}P(y^{\prime}\mid x^{\prime};\theta) ~ \color{red}{P(\theta\mid D)}~\mathrm{d}\theta
$$

如何计算后验$P(\theta|D)$是问题的关键.若$\theta \in \mathbf{R}^{d}$

$$
\begin{eqnarray}
P(\theta \mid D) & = & \frac{P(D \mid \theta) P(\theta)}{P(D)}  =  \frac{P(D \mid \theta) p(\theta)}{\int_{\theta} P(D, \theta) d \theta} \\ 
& = & \int_{\theta_{1}}\int_{\theta_{2}}\cdots \int_{\theta_{d}}P(D,\theta)~\mathrm{d}\theta_{1}~\mathrm{d}\theta_{1}~\cdots \mathrm{d}\theta_{d}
\end{eqnarray}
$$

若$~\theta~$维度很高，问题是Intractable的。

## 4.2 Approximate Inference

### 4.2.1 MCMC

$$
\begin{align}
\hat{y} &= \int_{\theta}P(y^{\prime}\mid x^{\prime};\theta) ~ \color{red}{P(\theta\mid D)}~\mathrm{d}\theta \\
&= \mathbb{E}_{P(\theta|D)}\left[ P(y^{\prime}\mid x^{\prime};\theta) \right] \\
& \approx \frac{1}{S}\sum_{s=1}^{S}P(y^{\prime}\mid x^{\prime};\theta^{s}) \\
\theta^{s} & \sim P(\theta|D)
\end{align}
$$

# 5. LDA（Latent Dirichlet Allocation）

<img src='attachment/69. LDA模型.png' style='zoom:50%'/>

LDA是生成模型，记Doc-Topic为$~\theta$，Topic-word 为$~\phi$，则文章的生成过程为

若文章包含$~n~$个单词，主题分布为$(0.6, 0.2, 0.1, 0.1)$，则生成过程为:

$$
for \quad i=1,2,\cdots,n: \\
\quad Z_{i} \sim \text{Mult}(n,p) \\
\quad w_{i} \sim \text{Mult}(\phi_{Z_{i}})
$$

<img src='attachment/69. LDA生成文本过程.png' style='zoom:50%'/>

$$
\begin{array}{l}
\theta_{i}: \text{Topic Distribution of Document} ~ i\\
\phi_{k}: \text{Word Distribution Under Topic} ~ k \\
Z_{ij}: \text{topic of jth word in Document} ~ i \\
w_{ij}: word/observation  \\
\alpha, \beta : prior(known)\\
\end{array} 
$$

$\text{Model Parameter:}~ \theta, \phi$ 

$\text{Hyperparameter:}~ \alpha, \beta$

$\text{latent parameter:}~z$

$\text{Observation:}~x$

### Step1: 根据先验$~\alpha$ 生成 $\theta_{i}$

$$
\begin{array}{}
for \quad i=1,2,\cdots,N \quad // ~ \text{iterate each doc} \\
\quad \quad \theta_{i} \longleftarrow \alpha
\end{array}
$$

$\theta_{i}= (\theta_{i1}, \theta_{i2}, \cdots, \theta_{iK})~$的特点:  
1. $\theta_{ij} \geq 0$  


2. $\sum_{j=1}^{K}\theta_{ij}=1$


$\theta_{i} \sim \text{Dir}(\alpha)$ 

i.e; &nbsp; $k=4, \alpha=(0.1, 0.1, 0.1, 0.1) \quad \theta_{i}=(0.5, 0.3, 0.1, 0.1)$

### Step2: 根据$\theta_{i}$确定第$~j~$个word的话题$Z_{ij}$

$$
Z_{ij} \sim \text{Mult}(\theta_{i})
$$

### Step3: 根据超参数$\beta$生成$\phi_{k}$

$$
\begin{array}{}
for \quad k=1,2,\cdots,K \quad // ~ \text{iterate each topic} \\
\quad \quad \phi_{k} \longleftarrow \beta
\end{array}
$$

$\phi_{k}=(\phi_{k1}, \phi_{k2}, \cdots, \phi_{k|V|})$的特点：
1. $\phi_{kj} \geq 0$ 


2. $\sum_{j=1}^{|V|}\phi_{kj}$=1

$\phi_{k} \sim \text{Dir}(\beta) \quad \beta\in \mathbf{R}^{|V|}$

### Step4: 根据$Z_{ij}, \phi$生成单词$w_{ij}$

$$
w_{ij} \sim \text{Mult}(\phi_{Z_{ij}})
$$

# 6. Gibbs Sampling

$\text{Hyperparameter:}~ \alpha, \beta$ 

$\text{Model Params:}~ \theta,\phi$

$\text{Hidden Vars:}~ Z$ 

$\text{Observations:}~ w$

Bayesian Inference: $P(\theta, \phi, Z|\alpha, \beta, w)$

### 6.1 Gibbs Sampling:

1. $\theta \sim P(\theta|w, \alpha, \beta, \phi, Z)$

2. $\phi \sim P(\phi|w, \alpha, \beta, \theta, Z)$

3. $Z \sim P(Z|w, \alpha, \beta, \theta, \phi)$

具体地，

$$
\begin{array}{l}
for \quad i=1,2,\cdots,N \\
\quad \quad \theta_{i} \sim P(\theta_{i}|w, \alpha, \beta, \phi, Z, \theta_{-i}) \\
\quad \\
for \quad k=1,2,\cdots,K \\
\quad \quad  \phi_{k} \sim P(\phi_{k}|w, \alpha, \beta, \theta, Z, \phi_{-k}) \\
\quad \\
for \quad i=1,2,\cdots,N \\
\quad for \quad j=1,2,\cdots,N_{i} \\
\quad \quad \quad Z_{ij} \sim P(Z_{ij}|w, \alpha, \beta, \theta, \phi, Z_{-ij})
\end{array}
$$

### 6.2 Markov Blanket（马尔科夫毯）

<img src='attachment/69. Markov Blanket.jpg' style='zoom:50%'/>

对于上面的贝叶斯网络，有  

$$
P(a|b,c,d,e,f,h,j,k,l,m) = P(a|b,c,d,e,f)
$$

### 6.3 $\theta_{i}$的后验分布

对于$~\theta_{i}$，有

$$
\begin{align}
\theta_{i} & \sim P(\theta_{i}|w, \alpha, \beta, \phi, Z, \theta_{-i}) \\
&= P(\theta_{i}|\alpha, Z) \quad(markov ~ blanket) \\
&= P(\theta_{i}|\alpha, \left\{Z_{ij}\right\}_{j=1}^{N_{i}}) \\
&= \frac{P\left(\theta_{i},\alpha, \left\{Z_{ij}\right\}_{j=1}^{N_{i}}\right)}{P\left(\alpha, \left\{Z_{ij}\right\}_{j=1}^{N_{i}}\right)} \\
&= \frac{P(\alpha)\cdot P(\theta_{i}|\alpha)\cdot P\left(\left\{Z_{ij}\right\}|\theta_{i}\right)}{P\left(\alpha, \left\{Z_{ij}\right\}_{j=1}^{N_{i}}\right)}\\
& \propto \underset{prior}{P(\theta_{i}|\alpha)}\cdot \underset{likelihood}{P\left(\left\{Z_{ij}\right\}|\theta_{i}\right)}
\end{align}
$$

$$
\begin{align}
&\quad \underset{Dirichlet}{P(\theta_{i}|\alpha)}\cdot \underset{Mult}{P\left(\left\{Z_{ij}\right\}|\theta_{i}\right)} \\
&= \frac{1}{B(\alpha)}\prod_{k=1}^{K}\theta_{ik}^{\alpha_{k}~-1} \cdot \prod_{j=1}^{N_{i}}\prod_{k=1}^{K}\theta_{ik}^{I(Z_{ij}~=k)} \\
&= \frac{1}{B(\alpha)} \prod_{k=1}^{K} \theta_{ik}^{\sum_{j=1}^{N_{~i}}~I(Z_{ij}~=k)+\alpha_{k}-1}
\end{align}
$$

因此有，

$$
P(\theta_{i} | \alpha) \sim \text{Dir}\left(\alpha+\sum_{j=1}^{N_{i}}I(Z_{ij}=k)\right)
$$

### 6.4 共轭分布

先验: $P(\theta_{i}|\alpha) \sim \text{Dir}(\alpha)$ 

likehood: $P(\left\{ Z_{ij} \right\}|\theta_{i}) \sim \text{Mult}(\theta_{i})$ 

后验: $P(\theta_{i} | \alpha) \sim \text{Dir}\left(\alpha+\sum_{j=1}^{N_{i}}I(Z_{ij}=k)\right)$

#### $Z_{ij}$的后验分布

<img src='attachment/69. LDA生成文本过程.png' style='zoom:50%'/>

$$
\begin{align}
&\quad P(Z_{ij} = k|w, \alpha, \beta, \phi, \theta, Z_{-ij}) \\
\quad \\
&= P(Z_{ij} = k|w_{ij}, \phi_{k}, \theta_{i}) \\
\quad \\
&= \frac{P(Z_{ij} = k, w_{ij}, \phi_{k}, \theta_{i})}{P(w_{ij}, \phi_{k}, \theta_{i})} \\
\quad \\
&= \frac{P(\theta_{i})P(Z_{ij}=k|\theta_{i})P(w_{ij}|Z_{ij}=k,\phi_{k})}{P(w_{ij}, \phi_{k}, \theta_{i})} \\
\quad \\
&\propto P(Z_{ij}=k|\theta_{i})\cdot P(w_{ij}|\phi_{k}) \\
\quad \\
&= \theta_{ik} \cdot \phi_{k,w_{ij}}
\end{align}
$$

因此，

$$
P(Z_{ij} = k|w, \alpha, \beta, \phi, \theta, Z_{-ij}) = \frac{\theta_{ik} \cdot \phi_{k,w_{ij}}}{\sum_{k}\theta_{ik} \cdot \phi_{k,w_{ij}}}
$$

### 6.5 $\phi_{k}$的后验分布

$$
\begin{align}
\phi_{k} &\sim P(\phi_{k}|w, \alpha, \beta, \theta, Z, \phi_{-k}) \\
\quad \\
& = P(\phi_{k}|w_{ij}, \beta,  Z_{ij}) \\
\quad \\
& = \frac{P(\phi_{k}, w_{ij}, \beta,  Z_{ij})}{P(w_{ij}, \beta,  Z_{ij})} \\
\quad \\
& = \frac{P(\beta)\cdot P(\phi_{k}|\beta)\cdot P(w_{ij}|Z_{ij}, \phi_{k})}{P(w_{ij}, \beta,  Z_{ij})} \\
\quad \\
& \propto  \underset{prior}{P(\phi_{k}|\beta)} \cdot \underset{likelihood}{P(w_{ij}|\phi_{k})} \\
\quad \\
& = \frac{1}{B(\beta)}\prod_{v=1}^{|V|}\phi_{k,v}^{\beta_{v}-1} \cdot \prod_{i=1}^{N}\prod_{j=1}^{N_{i}}\prod_{v=1}^{|V|}\phi_{k,v}^{I(w_{ij}~=v)} \\
\quad \\
& = \frac{1}{B(\beta)}\prod_{v=1}^{|V|}\phi_{k,v}^{\sum_{i=1}^{N}\sum_{j=1}^{N_{~i}}~I(w_{ij}~=v)+\beta_{v}-1}
\end{align}
$$

因此有，

$$
P(\phi_{k}|w,\alpha,\theta, Z, \phi_{-k}) = \text{Dir}\left(\beta + \sum_{i=1}^{N} \sum_{j=1}^{N_{i}}I(w_{ij}=v)\right)
$$

### 6.6 Gibbs Sampling Example 

$$
\begin{array}{l}
\text{for} \quad i=1,2,\cdots,N \\
\quad \quad \theta_{i} \sim P(\theta_{i}|w, \alpha, \beta, \phi, Z, \theta_{-i}) \\
\quad \\
\text{for} \quad k=1,2,\cdots,K \\
\quad \quad  \phi_{k} \sim P(\phi_{k}|w, \alpha, \beta, \theta, Z, \phi_{-k}) \\
\quad \\
\text{for} \quad i=1,2,\cdots,N \\
\quad \text{for} \quad j=1,2,\cdots,N_{i} \\
\quad \quad \quad Z_{ij} \sim P(Z_{ij}|w, \alpha, \beta, \theta, \phi, Z_{-ij})
\end{array}
$$

假设我们有两篇文档: 

<img src='69. Gibbs Sampling.png' style='zoom:50%'/>

#### Step1: 初始化

假设一共有3个主题，共计4个单词，随机初始化$\left\{ Z_{ij} \right\}$，初始化为

$K=3,~ \alpha=(0.1, 0.1, 0.1),~ \beta=(0.01, 0.01, 0.01, 0.01)$

$Z_{11}=1,~ Z_{12}=2, ~Z_{12}=2, ~Z_{14}=3$  
$Z_{21}=3,~ Z_{22}=1, ~Z_{23}=1, ~Z_{24}=2$

词库: (我们，今天，上课，天气)

#### Step2: 采样$~\theta_{i}~(\theta_{1}, \theta_{2})$

$\theta_{i} \sim \text{Dir}\left(\alpha + \sum_{j=1}^{N_{i}}I(Z_{ij}=k)\right)$

$\theta_{i} \sim \text{Dir}\left(\alpha_{1} + n_{1}, \alpha_{2} + n_{2}, \alpha_{3} + n_{3})\right)\quad n_{i}:$ # of words belong to topic k.

$\therefore \theta_{i} \sim \text{Dir}(0.1+1, 0.1+2, 0.1+1)$

$\therefore \theta_{i} \sim \text{Dir}(1.1, 2.1, 1.1)$，采样得到 

$\theta_{1}=(0.2, 0.7, 0.1)$

同理，可以采样得到

$\theta_{2}=(0.7, 0.25, 0.05)$

#### Step3: 采样$~\phi_{k}~(\phi_{1}, \phi_{2}, \phi_{3})$

$\phi_{k} \sim \text{Dir}\left( \beta + \sum_{i=1}^{N}\sum_{j=1}^{N_{i}}I(w_{ij}=v)\right)$

$\phi_{k} \sim \text{Dir}\left( \beta_{1} + m_{1},~ \beta_{2} + m_{2},~ \beta_{3} + m_{3},~ \beta_{4} + m_{4}\right) \quad m_{i}:$ 词库里的第$~i~$个单词有多少次属于Topic $k$. 

$\phi_{1} \sim \text{Dir}\left( 0.01+2,~ 0.01+0,~ 0.01+0,~ 0.01+1 \right)$，采样后得到$~\phi_{1}$

$\phi_{1} = (0.8,~ 0.05,~ 0.05,~ 0.1)$

$\phi_{2} \sim \text{Dir}\left( 0.01+0,~ 0.01+2,~ 0.01+1,~ 0.01+0 \right)$，采样后得到$~\phi_{2}$

$\phi_{2} = (0.05,~ 0.75,~ 0.15,~ 0.05)$

$\phi_{3} \sim \text{Dir}\left( 0.01+1,~ 0.01+1,~ 0.01+0,~ 0.01+0 \right)$，采样后得到$~\phi_{3}$

$\phi_{3} = (0.45,~ 0.35,~ 0.15,~ 0.05)$


#### Step4: 采样$~Z_{ij}$

$$
P(Z_{ij}=k|\alpha, \beta, w, \theta, \phi) \propto \theta_{ik}\cdot \phi_{k,w_{ij}}
$$

对于 $Z_{11}(我们)$

$P(Z_{11}=1|-) \propto \theta_{11}\cdot \phi_{1,我们} \quad = 0.2*0.8=0.16$

$P(Z_{11}=2|-) \propto \theta_{12}\cdot \phi_{2,我们} \quad = 0.7*0.05=0.035$

$P(Z_{11}=3|-) \propto \theta_{13}\cdot \phi_{3,我们} \quad = 0.1*0.45=0.045$

$\therefore \quad P(Z_{11}=1|-)=\frac{0.16}{0.16+0.035+0.045} \quad P(Z_{11}=2|-)=\frac{0.035}{0.16+0.035+0.045}\quad P(Z_{11}=3|-)=\frac{0.045}{0.16+0.035+0.045}$

$\therefore \quad Z_{11} \sim \text{Mult}(\frac{0.16}{0.24},~ \frac{0.035}{0.24},~ \frac{0.045}{0.24})$ 采样得到新的$Z_{11}$

同理，采样得到$Z_{12}, ~Z_{13}, ~Z_{14}, ~Z_{21}, ~Z_{22}, ~Z_{23}, ~Z_{24}$

### 6.7 MCMC的两个阶段

MCMC的两个阶段: 

1. Burn-in Period

2. Collecting Samples

6.6 节中介绍了一个迭代的采样过程，通过一次 Iteration，采样得到  

$\theta^{(i)}, ~ \phi^{(i)}, ~Z^{(i)}$

比如进行2000次Iteration，可能前1000次处于Burn-in阶段，而后进入Collting阶段，如何估计Burn-in的迭代次数，可以通过Perplexity曲线进行判断。

<img src='attachment/69. Perplexity.png' style='zoom:50%'/>

$$
\begin{align}
\theta_{Final} &= \frac{1}{1000}\left(\theta^{(1001)}+\cdots+\theta^{(2000)}\right) \\
\phi_{Final} &= \frac{1}{1000}\left(\phi^{(1001)}+\cdots+\phi^{(2000)}\right) \\
Z_{Final} &= \frac{1}{1000}\left(Z^{(1001)}+\cdots+Z^{(2000)}\right) \\
\end{align}
$$

Perplexity = $2^{-x}$，$x:$ average log likelihood.对于LDA，likelihood:

$$
\begin{align}
&\quad P(w_{ij}|\theta_{i}, \phi) \\
&= \sum_{k}P(Z_{ij}=k|\theta_{i})\cdot P(w_{ij}|Z_{ij}=k,\phi) \\
&= \sum_{k}P(Z_{ij}=k|\theta_{i})\cdot P(w_{ij}|\phi_{k}) 
\end{align}
$$